In [ ]:
import pandas as pd
from joblib import Parallel, delayed

In [ ]:
df = pd.read_pickle('./data/all_session_ids.pkl.gz')
df.shape

# Random samples

## Set 1

In [ ]:
sample_ids = df.sample(frac=0.01)

In [ ]:
sample_ids.shape

In [ ]:
sample_ids_dict = sample_ids.groupby('filepath')['session_id'].apply(lambda x: list(x)).to_dict()

In [ ]:
def load_one_file(x, y):
    return pd.read_csv(x)\
    .loc[lambda k: k.session_id.isin(y)]\
    .copy()

In [ ]:
sample_data = Parallel(n_jobs=14)(delayed(load_one_file)(filepath, one_sample_set)
                                 for filepath, one_sample_set in sample_ids_dict.items())

In [ ]:
sample_data = pd.concat(sample_data)

In [ ]:
sample_data.shape

In [ ]:
sample_data.to_pickle('./data/training_set_sample_data_1.pkl')

## Set 2

In [ ]:
sample_ids = df\
    .loc[lambda x: ~x.session_id.isin(sample_ids.session_id.tolist())]\
    .sample(n=1249507)
sample_ids.shape

In [ ]:
sample_ids_dict = sample_ids.groupby('filepath')['session_id'].apply(lambda x: list(x)).to_dict()

In [ ]:
sample_data = Parallel(n_jobs=14)(delayed(load_one_file)(filepath, one_sample_set)
                                 for filepath, one_sample_set in sample_ids_dict.items())
sample_data = pd.concat(sample_data)

In [ ]:
sample_data.to_pickle('./data/training_set_sample_data_2.pkl')

# Create samples of particular length

Note: Overall random sample set 2 is not being included in the following samples

In [ ]:
sample_ids = pd.read_pickle('./data/training_set_sample_ids_2.pkl.gz')

In [ ]:
for a_len in range(10, 21):
    this_sample_ids = df\
        .loc[lambda x: ~x.session_id.isin(sample_ids.session_id.tolist())]\
        .loc[lambda x: x.session_length == str(a_len)]\
        .sample(n=2499014)
    sample_ids_dict = this_sample_ids.groupby('filepath')['session_id'].apply(lambda x: list(x)).to_dict()
    sample_data = Parallel(n_jobs=14)(delayed(load_one_file)(filepath, one_sample_set)
                                      for filepath, one_sample_set in sample_ids_dict.items())
    sample_data = pd.concat(sample_data)
    sample_data.to_pickle('./data/final_samples/training_set_sample_data_l{}.pkl'.format(a_len))